In [ ]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2021-08-31 11:58:48--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2021-08-31 11:58:49--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc27d46499a9e2d53dc6573293a4.dl.dropboxusercontent.com/cd/0/inline/BVQcJufiovAiRB2AsL6EplKlyRc2qEI4mYm13ISEGtbHimesjqU1MklpcAlsa_cV0tklOEBz7qGGYWOBq3iBpvUM1NyQWtnZKJ_9OxXgN9QtWirnWPhEWLdXwfn-K1DbzmSfFE_uG7gHul6BtNOLNae7/file# [following]
--2021-08-31 11:58:49--  https://uc27d46499a9e2d53dc6573293a4.dl.dropboxusercontent.com/cd/0/inline/BVQcJufiovAiRB2AsL6EplKlyRc2qEI4mYm13ISEGtbHimesjqU

In [ ]:
# Let's unzip the file
!unzip -q "/content/movie-reviews-dataset.zip"

In [ ]:
from tensorflow.keras.preprocessing import  text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, GRU, Embedding, Dropout

In [ ]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
      lambda text, label: (regex_replace(text, "<br />", " "), label),
  )

In [ ]:
train_data = prepareData("/content/movie-reviews-dataset/train")
test_data = prepareData("/content/movie-reviews-dataset/test")

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b'Amongst the standard one liner type action films, where acting and logic are checked at the door, this movie is at the top of the class. If the person in charge of casting were to have put "good" actors in this flick, it would have been worse(excepting Richard Dawson who actually did act well, if you can call playing yourself "acting"). I love this movie! The Running Man is in all likelihood God\'s gift to man(okay maybe just men). Definitely the most quotable movie of our time so I\'ll part you with my favorite line: "It\'s all part of life\'s rich pattern Brenda, and you better F*****g get used to it." Ahh, more people have been called "Brenda" for the sake of quoting this film than I can possibly imagine.'
1


In [ ]:
model =Sequential()

In [ ]:
model.add(Input(shape=(1,), dtype="string"))

In [ ]:
max_tokens= 1000
max_len = 100
vectorize_layer = TextVectorization(
    max_tokens = max_tokens,
    output_mode = 'int',
    output_sequence_length = max_len,
)

In [ ]:
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

In [ ]:
model.add(Embedding(max_tokens + 1, 128))

model.add(GRU(64))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 61s 75ms/step - loss: 0.5422 - accuracy: 0.7136
Epoch 2/10
782/782 [==============================] - 60s 77ms/step - loss: 0.4204 - accuracy: 0.8065
Epoch 3/10
782/782 [==============================] - 60s 76ms/step - loss: 0.3862 - accuracy: 0.8228
Epoch 4/10
782/782 [==============================] - 60s 76ms/step - loss: 0.3678 - accuracy: 0.8316
Epoch 5/10
782/782 [==============================] - 59s 75ms/step - loss: 0.3476 - accuracy: 0.8394
Epoch 6/10
782/782 [==============================] - 59s 75ms/step - loss: 0.3295 - accuracy: 0.8480
Epoch 7/10
782/782 [==============================] - 58s 75ms/step - loss: 0.3115 - accuracy: 0.8573
Epoch 8/10
782/782 [==============================] - 58s 75ms/step - loss: 0.2916 - accuracy: 0.8665
Epoch 9/10
782/782 [==============================] - 60s 77ms/step - loss: 0.2743 - accuracy: 0.8735
Epoch 10/10
782/782 [==============================] - 100s 128ms/step - loss: 0.2

In [ ]:
model.evaluate(test_data)

782/782 [==============================] - 32s 40ms/step - loss: 0.6414 - accuracy: 0.7632


[0.6413635015487671, 0.7632399797439575]

In [ ]:
text = "This movis is bad"

In [ ]:
model.predict([text]) # 0 = Negative review, 1 = positive review

array([[0.03322583]], dtype=float32)

In [ ]:
text2 = "This movis is good"

In [ ]:
model.predict([text2]) # 0 = Negative review, 1 = positive review

array([[0.96662104]], dtype=float32)

LSTM and GRU networks are performing better on the dataset that than the RNN. 